In [29]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Mount Google Drive
drive.mount('/content/drive')

# Path of folder
folder_path = '/content/drive/MyDrive/Colab Notebooks/2024_08_05/'

# Get list of all CSV files in the folder
csv_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

# Initialize an empty list to hold the dataframes
dfs = []

# Loop through the CSV files and read each one into a dataframe
for file in csv_files:
    df = pd.read_csv(file)
    dfs.append(df)

# Concatenate all dataframes into a single dataframe
data = pd.concat(dfs, ignore_index=True)

# Sort by 'First Occurred On' in descending order (latest alarms first)
data = data.sort_values(by='First Occurred On', ascending=False)

# Convert 'First Occurred On' to datetime
data['First Occurred On'] = pd.to_datetime(data['First Occurred On'], errors='coerce')

# Reference time
reference_time = pd.to_datetime('2024-07-31 08:00:00')

# Calculate the relative day index
data['Relative Day Index'] = ((data['First Occurred On'] - reference_time).dt.total_seconds() // 86400).fillna(-9999).astype(int)

# Adjust the index to set the 08:00 to 08:00 of the next day as 0, and previous days as -1, -2, -3, etc.
data['Relative Day Index'] = data['Relative Day Index'].apply(lambda x: x if x < 0 else x)

# Filter data to include only rows before the 0 Relative Day Index
data_before_0 = data[data['Relative Day Index'] < 0]

# Keep only relevant columns
columns_to_keep = ['Site ID', 'Alarm Name', 'Device Type', 'Relative Day Index']
data_before_0 = data_before_0[columns_to_keep]

# Filter the dataset to focus on specific alarms of interest
alarms_of_interest = ['Mains Failure Alarm', 'Battery Deep Discharge Alarm',
                      'NE Is Disconnected', 'Heartbeat Failure',
                      'Cell Out of Service', 'Cell Unavailable']

filtered_data = data_before_0[data_before_0['Alarm Name'].isin(alarms_of_interest)]

# Encode categorical variables
le_site = LabelEncoder()
le_device = LabelEncoder()
le_alarm = LabelEncoder()

filtered_data['Site ID Encoded'] = le_site.fit_transform(filtered_data['Site ID'])
filtered_data['Device Type Encoded'] = le_device.fit_transform(filtered_data['Device Type'])
filtered_data['Alarm Name Encoded'] = le_alarm.fit_transform(filtered_data['Alarm Name'])

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(filtered_data[['Site ID Encoded', 'Device Type Encoded', 'Alarm Name Encoded', 'Relative Day Index']])

# Prepare the data for LSTM (reshaping to 3D for LSTM)
def create_lstm_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step), :-1])  # Features up to the last column
        y.append(data[i + time_step, -1])  # Target is the next alarm name
    return np.array(X), np.array(y)

time_step = 7  # Number of past days to consider for prediction
X, y = create_lstm_dataset(scaled_data, time_step)

# Reshape X to be [samples, time steps, features]
X = X.reshape((X.shape[0], time_step, X.shape[2]))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(time_step, X.shape[2])))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))  # Predict the encoded Alarm Name

model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, batch_size=64, epochs=10, verbose=1)

# Predict on the test set
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)  # Round and convert to integer

# Inverse transform to get the original Alarm Names
y_pred_decoded = le_alarm.inverse_transform(y_pred.flatten())
y_test_decoded = le_alarm.inverse_transform(y_test.flatten())

# Evaluate the model
print("Predicted Alarm Names:\n", y_pred_decoded[:10])
print("True Alarm Names:\n", y_test_decoded[:10])

# Predict alarms for the next 7 days using the LSTM model
def predict_upcoming_alarms_lstm(model, X_test, le_alarm, days=7):
    predictions = []
    X_input = X_test[-1:]  # Start with the last input sequence

    for day in range(1, days + 1):
        day_prediction = model.predict(X_input)
        day_prediction = np.round(day_prediction).astype(int)
        decoded_prediction = le_alarm.inverse_transform(day_prediction.flatten())

        predictions.append((X_input[0, -1, -1] + day, decoded_prediction[0]))  # Adjust Relative Day Index and append

        # Update X_input with the new prediction for the next iteration
        new_input = np.hstack((X_input[:, 1:, :], np.expand_dims(day_prediction, axis=2)))
        X_input = new_input

    return predictions

# Use the trained model to predict alarms for the next 7 days
upcoming_predictions = predict_upcoming_alarms_lstm(model, X_test.copy(), le_alarm)

# Print predictions
for day_index, alarm in upcoming_predictions:
    print(f"Day Index: {day_index}, Alarm: {alarm}")

    # You can add back the Site ID and Device Type decoding for a more complete output
    print(f"Site ID: {le_site.inverse_transform([X_test[0, 0, 0]])[0]}, Device Type: {le_device.inverse_transform([X_test[0, 0, 1]])[0]}")



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-29-6fb954c3a36f>:26: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
<ipython-input-29-6fb954c3a36f>:26: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
<ipython-input-29-6fb954c3a36f>:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Site ID Encoded'] = le_site.fit_transform(filtered_data['Site ID'])
<ipython-input-29-6fb954c3a36f>:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

Epoch 1/10
11595/11595 ━━━━━━━━━━━━━━━━━━━━ 138s 12ms/step - loss: 0.0802
Epoch 2/10
11595/11595 ━━━━━━━━━━━━━━━━━━━━ 135s 12ms/step - loss: 0.0764
Epoch 3/10
11595/11595 ━━━━━━━━━━━━━━━━━━━━ 135s 12ms/step - loss: 0.0730
Epoch 4/10
11595/11595 ━━━━━━━━━━━━━━━━━━━━ 146s 12ms/step - loss: 0.0701
Epoch 5/10
11595/11595 ━━━━━━━━━━━━━━━━━━━━ 143s 12ms/step - loss: 0.0691
Epoch 6/10
11595/11595 ━━━━━━━━━━━━━━━━━━━━ 143s 12ms/step - loss: 0.0685
Epoch 7/10
11595/11595 ━━━━━━━━━━━━━━━━━━━━ 141s 12ms/step - loss: 0.0679
Epoch 8/10
11595/11595 ━━━━━━━━━━━━━━━━━━━━ 142s 12ms/step - loss: 0.0674
Epoch 9/10
11595/11595 ━━━━━━━━━━━━━━━━━━━━ 144s 12ms/step - loss: 0.0669
Epoch 10/10
11595/11595 ━━━━━━━━━━━━━━━━━━━━ 198s 12ms/step - loss: 0.0666
5798/5798 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step


ValueError: y contains previously unseen labels: [0.00568182 0.01136364 0.01704545 0.02272727 0.02840909 0.03409091
 0.03977273 0.04545455 0.05113636 0.05681818 0.0625     0.06818182
 0.07386364 0.07954545 0.08522727 0.09090909 0.09659091 0.10227273
 0.10795455 0.11363636 0.11931818 0.125      0.13068182 0.13636364
 0.14204545 0.14772727 0.15340909 0.15909091 0.16477273 0.17045455
 0.17613636 0.18181818 0.1875     0.19318182 0.19886364 0.20454545
 0.21022727 0.21590909 0.22159091 0.22727273 0.23295455 0.23863636
 0.24431818 0.25       0.25568182 0.26136364 0.26704545 0.27272727
 0.27840909 0.28409091 0.28977273 0.29545455 0.30113636 0.30681818
 0.3125     0.31818182 0.32386364 0.32954545 0.33522727 0.34090909
 0.34659091 0.35227273 0.35795455 0.36363636 0.36931818 0.375
 0.38068182 0.38636364 0.39204545 0.39772727 0.40340909 0.40909091
 0.41477273 0.42045455 0.42613636 0.43181818 0.4375     0.44318182
 0.44886364 0.45454545 0.46022727 0.46590909 0.47159091 0.47727273
 0.48295455 0.4886